In [ ]:
import pandas as pd
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
csv_file_path = '/content/drive/MyDrive/School/High-risk Project/Sepsis DataSet/'
!ls '/content/drive/MyDrive/School/High-risk Project/Sepsis DataSet'

ADMISSIONS.csv				  SEPSIS_POS_CHART_EVENTS_AGE_ABOVE_64.csv
clean_sepsis_neg.csv			  SEPSIS_POS_CHART_EVENTS.csv
clean_sepsis_pos.csv			  sepsis_positive_pivoted.csv
DIAGNOSES_ICD.csv			  SEPSIS_POS_LAB_EVENTS_AGE_ABOVE_64-1714062992998.csv
Sepsis_Analysis.sql			  SEPSIS_POS_LAB_EVENTS_AGE_ABOVE_64.csv
SEPSIS_NEG_CHART_EVENTS_AGE_ABOVE_64.csv  SEPSIS_POS_PAT_AGE.csv
SEPSIS_NEG_CHART_EVENTS.csv		  test_data.csv
SEPSIS_NEG_LAB_EVENTS_AGE_ABOVE_64.csv	  train_data.csv
SEPSIS_NEG_PAT_AGE.csv			  validate_data.csv
sepsis_neg_pivoted.csv


In [ ]:
### UPDATE MACROS
# MACROS - Update these as needed
TRAIN_DATASET  = csv_file_path + 'train_data.csv'
VALIDATE_DATASET = csv_file_path + 'validate_data.csv'
LLM_PROMPT        = 'Does this patient most likely have sepsis? Reply only with a Yes or a No as your answer.'
# START_PROMPT      = 'Determine whether this patient has sepsis based on the listed medical data. Based on the highest probability, does this patient have sepsis? You must reply with only one word: Yes or No.'

# Imports
import csv
import json
from google.colab import files

In [ ]:
TRAIN_DATASET

'/content/drive/MyDrive/School/High-risk Project/Sepsis DataSet/train_data.csv'

In [ ]:
VALIDATE_DATASET

'/content/drive/MyDrive/School/High-risk Project/Sepsis DataSet/validate_data.csv'

In [ ]:
class patientData(object):
  'This class represents the patient data. It allows us to track for sepsis or not'
  def __init__(self, medicalData, sepsis):
    self.medicalData = medicalData
    self.sepsis = sepsis

In [ ]:
def deserializeCSV(fileName):
  'This function will either return a new dataset or append to a current dataset if you pass in the previous one'
  data = []
  # Open CSV file as csvf
  with open(fileName, encoding='utf-8') as csvf:
    csvReader = csv.DictReader(csvf)

    # Convert each row into a dictionary
    # and add it to data
    for row in csvReader:
      formattedString = ''

      # Grab data
      heart_rate              = row.get('Heart Rate')
      # blood_pressure_mean     = row.get('Invasive Blood Pressure mean')
      respiratory_rate        = row.get('Respiratory Rate (RR)')
      temperature             = row.get('Temperature (Fahrenheit)')
      white_blood_cells       = row.get('White Blood Cells (cells/mm^3)')
      target                  = row.get('target')

      # Format data into string and store into results
      # formattedString = "- Heart Rate: " + heart_rate + " - Blood Pressure Mean: " + blood_pressure_mean + " - Respiratory Rate: " + respiratory_rate + " - Temperature: " + temperature + " - White Blood Cell Count: " + white_blood_cells
      formattedString = "The patient's heart rate is " + heart_rate + ", respiratory rate is " + respiratory_rate + ", temperature is " + temperature + ", and white blood cell count is " + white_blood_cells
      # Determine if patient is from negative or positive dataset
      if target == 'No':
        patient = patientData(formattedString, False)
        print("No")
      else:
        patient = patientData(formattedString, True)
        print("Yes")

      # Append the patient data
      data.append(patient)

    # Clean up by closing file and returning results
    csvf.close()
    return data

In [ ]:
# Deserialize CSV data
trainData = deserializeCSV(TRAIN_DATASET) # get initial dataset

In [ ]:
validateData = deserializeCSV(VALIDATE_DATASET) # get initial dataset

In [ ]:
print(len(trainData))
print(len(validateData))

361
77


In [ ]:
def serializeToJson(csvData):
  'This function will convert the patient data into the required jsonl format'
  # Create empty data container
  data = []

  # Format information for each patient
  for patient in csvData:
    messages_dict = {"messages":[]}
    messages_value = []

    # Add medical data to user content
    user      = {"role": "user", "content": patient.medicalData + '. ' + LLM_PROMPT}

    # Determine if patient has sepsis or not and format assistant
    if patient.sepsis == False:
      assistant = {"role": "assistant", "content": "No"}
    else:
      assistant = {"role": "assistant", "content": "Yes"}

    # Compile data
    messages_value.append(user)
    messages_value.append(assistant)
    messages_dict.update({"messages": messages_value})
    data.append(messages_dict)


  # Ship it
  return data

In [ ]:
# Serialize CSV data into json file
trainJSON = serializeToJson(trainData)
validateJSON = serializeToJson(validateData)

In [ ]:
trainJSON[0]

{'messages': [{'role': 'user',
   'content': "The patient's heart rate is 94.0 BPM, respiratory rate is 24.0 BPM, temperature is 95.2 Deg. F, and white blood cell count is 14.7 K/uL. Does this patient most likely have sepsis? Reply only with a Yes or a No as your answer."},
  {'role': 'assistant', 'content': 'Yes'}]}

In [ ]:
validateJSON[0]

{'messages': [{'role': 'user',
   'content': "The patient's heart rate is 77.0 BPM, respiratory rate is 18.0 BPM, temperature is 96.9 Deg. F, and white blood cell count is 13.3 K/uL. Does this patient most likely have sepsis? Reply only with a Yes or a No as your answer."},
  {'role': 'assistant', 'content': 'Yes'}]}

In [ ]:
with open("training_set.jsonl", 'w') as f:
    for item in trainJSON:
        f.write(json.dumps(item) + "\n")

    # Always close your files kids :)
    f.close()

with open("training_set.jsonl") as f:
    for line in f:
        print(line)

    # Always close your files kids :)
    f.close()

{"messages": [{"role": "user", "content": "The patient's heart rate is 94.0 BPM, respiratory rate is 24.0 BPM, temperature is 95.2 Deg. F, and white blood cell count is 14.7 K/uL. Does this patient most likely have sepsis? Reply only with a Yes or a No as your answer."}, {"role": "assistant", "content": "Yes"}]}

{"messages": [{"role": "user", "content": "The patient's heart rate is 82.0 BPM, respiratory rate is 16.0 BPM, temperature is 97.1 Deg. F, and white blood cell count is 7.4 K/uL. Does this patient most likely have sepsis? Reply only with a Yes or a No as your answer."}, {"role": "assistant", "content": "Yes"}]}

{"messages": [{"role": "user", "content": "The patient's heart rate is 128.0 BPM, respiratory rate is 21.0 BPM, temperature is 97.3 Deg. F, and white blood cell count is 18.9 K/uL. Does this patient most likely have sepsis? Reply only with a Yes or a No as your answer."}, {"role": "assistant", "content": "Yes"}]}

{"messages": [{"role": "user", "content": "The patient'

In [ ]:
with open("validation_set.jsonl", 'w') as f:
    for item in validateJSON:
        f.write(json.dumps(item) + "\n")

    # Always close your files kids :)
    f.close()

with open("validation_set.jsonl") as f:
    for line in f:
        print(line)

    # Always close your files kids :)
    f.close()

{"messages": [{"role": "user", "content": "The patient's heart rate is 77.0 BPM, respiratory rate is 18.0 BPM, temperature is 96.9 Deg. F, and white blood cell count is 13.3 K/uL. Does this patient most likely have sepsis? Reply only with a Yes or a No as your answer."}, {"role": "assistant", "content": "Yes"}]}

{"messages": [{"role": "user", "content": "The patient's heart rate is 81.0 BPM, respiratory rate is 24.0 BPM, temperature is 98.9 Deg. F, and white blood cell count is 9.4 K/uL. Does this patient most likely have sepsis? Reply only with a Yes or a No as your answer."}, {"role": "assistant", "content": "No"}]}

{"messages": [{"role": "user", "content": "The patient's heart rate is 64.0 BPM, respiratory rate is 14.0 BPM, temperature is 100.5 Deg. F, and white blood cell count is 9.1 K/uL. Does this patient most likely have sepsis? Reply only with a Yes or a No as your answer."}, {"role": "assistant", "content": "No"}]}

{"messages": [{"role": "user", "content": "The patient's h

In [ ]:
# Load the training set
with open('training_set.jsonl', 'r', encoding='utf-8') as f:
    training_dataset = [json.loads(line) for line in f]

# Training dataset stats
print("Number of examples in training set:", len(training_dataset))
print("First example in training set:")
for message in training_dataset[0]["messages"]:
    print(message)

# Load the validation set
with open('validation_set.jsonl', 'r', encoding='utf-8') as f:
    validation_dataset = [json.loads(line) for line in f]

# Validation dataset stats
print("\nNumber of examples in validation set:", len(validation_dataset))
print("First example in validation set:")
for message in validation_dataset[0]["messages"]:
    print(message)

Number of examples in training set: 361
First example in training set:
{'role': 'user', 'content': "The patient's heart rate is 94.0 BPM, respiratory rate is 24.0 BPM, temperature is 95.2 Deg. F, and white blood cell count is 14.7 K/uL. Does this patient most likely have sepsis? Reply only with a Yes or a No as your answer."}
{'role': 'assistant', 'content': 'Yes'}

Number of examples in validation set: 77
First example in validation set:
{'role': 'user', 'content': "The patient's heart rate is 77.0 BPM, respiratory rate is 18.0 BPM, temperature is 96.9 Deg. F, and white blood cell count is 13.3 K/uL. Does this patient most likely have sepsis? Reply only with a Yes or a No as your answer."}
{'role': 'assistant', 'content': 'Yes'}


In [ ]:
pip install tiktoken

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 11.7 MB/s eta 0:00:00


In [ ]:
import tiktoken
import numpy as np
from collections import defaultdict

In [ ]:
# Validate token counts

encoding = tiktoken.get_encoding("cl100k_base") # default encoding used by gpt-4, turbo, and text-embedding-ada-002 models

def num_tokens_from_messages(messages, tokens_per_message=3, tokens_per_name=1):
    num_tokens = 0
    for message in messages:
        num_tokens += tokens_per_message
        for key, value in message.items():
            num_tokens += len(encoding.encode(value))
            if key == "name":
                num_tokens += tokens_per_name
    num_tokens += 3
    return num_tokens

def num_assistant_tokens_from_messages(messages):
    num_tokens = 0
    for message in messages:
        if message["role"] == "assistant":
            num_tokens += len(encoding.encode(message["content"]))
    return num_tokens

def print_distribution(values, name):
    print(f"\n#### Distribution of {name}:")
    print(f"min / max: {min(values)}, {max(values)}")
    print(f"mean / median: {np.mean(values)}, {np.median(values)}")
    print(f"p5 / p95: {np.quantile(values, 0.1)}, {np.quantile(values, 0.9)}")

files = ['training_set.jsonl', 'validation_set.jsonl']

for file in files:
    print(f"Processing file: {file}")
    with open(file, 'r', encoding='utf-8') as f:
        dataset = [json.loads(line) for line in f]

    total_tokens = []
    assistant_tokens = []

    for ex in dataset:
        messages = ex.get("messages", {})
        total_tokens.append(num_tokens_from_messages(messages))
        assistant_tokens.append(num_assistant_tokens_from_messages(messages))

    print_distribution(total_tokens, "total tokens")
    print_distribution(assistant_tokens, "assistant tokens")
    print('*' * 50)

Processing file: training_set.jsonl

#### Distribution of total tokens:
min / max: 79, 79
mean / median: 79.0, 79.0
p5 / p95: 79.0, 79.0

#### Distribution of assistant tokens:
min / max: 1, 1
mean / median: 1.0, 1.0
p5 / p95: 1.0, 1.0
**************************************************
Processing file: validation_set.jsonl

#### Distribution of total tokens:
min / max: 79, 79
mean / median: 79.0, 79.0
p5 / p95: 79.0, 79.0

#### Distribution of assistant tokens:
min / max: 1, 1
mean / median: 1.0, 1.0
p5 / p95: 1.0, 1.0
**************************************************


In [ ]:
pip install openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.6/311.6 kB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 7.3 MB/s eta 0:00:00


In [ ]:
file_path = "/content/drive/MyDrive/School/Azure/azure_config.json"
json_file = open(file_path)
config_details = json.load(json_file)

In [ ]:
# Assign variables from json file
API_KEY = config_details['AZURE_OPEN_API_KEY']
API_BASE = config_details['ENDPOINT']
API_VERSION = config_details['API_VERSION']
DEPLOYMENT_NAME = config_details['DEPLOYMENT_NAME']
GPT_MODEL_NAME = config_details['GPT_MODEL_NAME']

In [ ]:
# Upload fine-tuning files
from openai import AzureOpenAI

client = AzureOpenAI(
  azure_endpoint = API_BASE,
  api_key = API_KEY,
  api_version = "2024-02-01"  # This API version or later is required to access fine-tuning for turbo/babbage-002/davinci-002
)

training_file_name = 'training_set.jsonl'
validation_file_name = 'validation_set.jsonl'

# Upload the training and validation dataset files to Azure OpenAI with the SDK.

training_response = client.files.create(
    file = open(training_file_name, "rb"), purpose="fine-tune"
)
training_file_id = training_response.id

validation_response = client.files.create(
    file = open(validation_file_name, "rb"), purpose="fine-tune"
)
validation_file_id = validation_response.id

print("Training file ID:", training_file_id)
print("Validation file ID:", validation_file_id)

Training file ID: file-208d46ff51814d5ca1ddc23104bb5886
Validation file ID: file-944d734fe84b438096b6868f314ecf8f


In [ ]:
# Submit fine-tuning training job

response = client.fine_tuning.jobs.create(
    training_file = training_file_id,
    validation_file = validation_file_id,
    model = "gpt-35-turbo-0613", # Enter base model name. Note that in Azure OpenAI the model name contains dashes and cannot contain dot/period characters.
    # make sure Resourse region is set to East US 2
    hyperparameters={
      "n_epochs": 7,
      "batch_size": 1,
      "learning_rate_multiplier": 0.1
    }
)

job_id = response.id

# You can use the job ID to monitor the status of the fine-tuning job.
# The fine-tuning job will take some time to start and complete.

print("Job ID:", response.id)
print("Status:", response.status)
print(response.model_dump_json(indent=2))

Job ID: ftjob-d5101b5950524d3ab0cff056384d653b
Status: pending
{
  "id": "ftjob-d5101b5950524d3ab0cff056384d653b",
  "created_at": 1714284629,
  "error": null,
  "fine_tuned_model": null,
  "finished_at": null,
  "hyperparameters": {
    "n_epochs": 7,
    "batch_size": 1,
    "learning_rate_multiplier": 0.1
  },
  "model": "gpt-35-turbo-0613",
  "object": "fine_tuning.job",
  "organization_id": null,
  "result_files": null,
  "seed": null,
  "status": "pending",
  "trained_tokens": null,
  "training_file": "file-208d46ff51814d5ca1ddc23104bb5886",
  "validation_file": "file-944d734fe84b438096b6868f314ecf8f",
  "integrations": null
}


In [ ]:
# Track training status

from IPython.display import clear_output
import time

start_time = time.time()

# Get the status of our fine-tuning job.
response = client.fine_tuning.jobs.retrieve(job_id)

status = response.status

# If the job isn't done yet, poll it every 10 seconds.
while status not in ["succeeded", "failed"]:
    time.sleep(10)

    response = client.fine_tuning.jobs.retrieve(job_id)
    print(response.model_dump_json(indent=2))
    print("Elapsed time: {} minutes {} seconds".format(int((time.time() - start_time) // 60), int((time.time() - start_time) % 60)))
    status = response.status
    print(f'Status: {status}')
    clear_output(wait=True)

print(f'Fine-tuning job {job_id} finished with status: {status}')

# List all fine-tuning jobs for this resource.
print('Checking other fine-tune jobs for this resource.')
response = client.fine_tuning.jobs.list()
print(f'Found {len(response.data)} fine-tune jobs.')

Fine-tuning job ftjob-d5101b5950524d3ab0cff056384d653b finished with status: succeeded
Checking other fine-tune jobs for this resource.
Found 3 fine-tune jobs.
